# Analysing school and standardised test data

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Calculate the total number of unique schools
unique_school_names = school_data_complete["school_name"].unique()
total_schools = len(unique_school_names)
total_schools

15

In [3]:
#Calculate the total number of students
total_students = school_data_complete["student_name"].count()
total_students

39170

In [4]:
#Calculate the total budget
total_budget = (school_data_complete["budget"].unique()).sum()
total_budget

24649428

In [5]:
#Calculate the average maths score
average_maths_score = school_data_complete["maths_score"].mean()
average_maths_score

70.33819249425581

In [6]:
#Calculate the average reading score
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score

69.98013786060761

In [7]:
#Calculate the percentage of students who passed maths (grade >= 50)
students_passing_maths = school_data_complete.loc[school_data_complete["maths_score"] >= 50]["maths_score"].count()
percent_passing_maths = (students_passing_maths / total_students) * 100
percent_passing_maths

86.07863160582077

In [8]:
#Calculate the percentage of students who passed reading (grade >= 50)
students_passing_reading = school_data_complete.loc[school_data_complete["reading_score"] >=50]["reading_score"].count()
percent_passing_reading = (students_passing_reading / total_students) * 100
percent_passing_reading

84.42685728874139

In [9]:
#Calculate the percentage who passed both maths and reading (% overall passing)
students_passing_both = school_data_complete.loc[(school_data_complete["maths_score"] >= 50) & (school_data_complete["reading_score"] >= 50)].count()["student_name"]
overall_passing = (students_passing_both / total_students) * 100
overall_passing


72.80827163645647

In [10]:
#Create a dataframe to summarise above results
LGA_summary = pd.DataFrame({"Total Schools": total_schools,
                                 "Total Students": f"{total_students:,}",
                                 "Total Budget": f"${total_budget:,.2f}",
                                 "Average Maths Score": f"{average_maths_score:.6f}",
                                 "Average Reading Score": f"{average_reading_score:.6f}",
                                 "% Passing Maths": f"{percent_passing_maths:.6f}",
                                 "% Passing Reading": f"{percent_passing_reading:.6f}",
                                 "% Overall Passing": f"{overall_passing:.6f}"}, index=[0])
LGA_summary

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [11]:
#Group by schools
school_names = school_data_complete.set_index("school_name").groupby(["school_name"])

In [12]:
#School types by school name
school_types = school_data.set_index("school_name")["type"]

In [13]:
#Total students
students_per_school = school_names["Student ID"].count()

In [14]:
#Total school budget
school_budget = school_data.set_index("school_name")["budget"]

In [15]:
#Per student budget
budget_per_student = (school_data.set_index("school_name")["budget"] / \
    school_data.set_index("school_name")["size"])

In [16]:
#Average maths score
average_maths_score = school_names["maths_score"].mean()

In [17]:
#Average reading score
average_reading_score = school_names["reading_score"].mean()

In [18]:
# % passing maths
passing_maths_percent = school_data_complete[school_data_complete\
    ["maths_score"] >= 50].groupby("school_name")["Student ID"].count()\
        / students_per_school * 100

In [19]:
# % passing reading
passing_reading_percent = school_data_complete[school_data_complete\
    ["reading_score"] >= 50].groupby("school_name")["Student ID"].count()\
        / students_per_school * 100

In [20]:
# % overall passing both maths and reading
overall_pass = school_data_complete[(school_data_complete\
    ["maths_score"] >= 50) & (school_data_complete["reading_score"] >= 50)]\
        .groupby("school_name")["Student ID"].count() / students_per_school * 100

In [21]:
#Create a dataframe to summarise above results
school_summary = pd.DataFrame({
    "School Type": school_types,
    "Total Students": students_per_school,
    "Per Student Budget": budget_per_student,
    "Total School Budget": school_budget,
    "Average Maths Score": average_maths_score,
    "Average Reading Score": average_reading_score,
    "% Passing Maths": passing_maths_percent,
    "% Passing Reading": passing_reading_percent,
    "% Overall Passing": overall_pass
})


#Rearrange columns
school_summary = school_summary[["School Type", 
                          "Total Students", 
                          "Total School Budget", 
                          "Per Student Budget", 
                          "Average Maths Score", 
                          "Average Reading Score",
                          "% Passing Maths",
                          "% Passing Reading",
                          "% Overall Passing"]]


#formatting
school_summary.style.format({"Total Students": '{:}',
                          "Total School Budget": "${:,.2f}",
                          "Per Student Budget": "${:.2f}",
                          "Average Maths Score": "{:6f}", 
                          "Average Reading Score": "{:6f}", 
                          "% Passing Maths": "{:6f}", 
                          "% Passing Reading": "{:6f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [22]:
top_performing = school_summary.sort_values("% Overall Passing", ascending = False)
top_performing.head().style.format({"Total Students": '{:}',
                           "Total School Budget": "${:,.2f}", 
                           "Per Student Budget": "${:.2f}", 
                           "% Passing Maths": "{:6f}", 
                           "% Passing Reading": "{:6f}", 
                           "% Overall Passing": "{:6f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,90.797699,87.396849,79.419855


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [23]:
bottom_performing = top_performing.tail()
bottom_performing = bottom_performing.sort_values('% Overall Passing')
bottom_performing.style.format({"Total Students": '{: }', 
                       "Total School Budget": "${:,.2f}", 
                       "Per Student Budget": "${:.2f}", 
                       "% Passing Maths": "{:6f}", 
                       "% Passing Reading": "{:6f}", 
                       "% Overall Passing": "{:6f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [24]:
# Create data series of scores by year levels using conditionals
year_nine = school_data_complete[(school_data_complete["year"] == 9)]
year_ten = school_data_complete[(school_data_complete["year"] == 10)]
year_eleven = school_data_complete[(school_data_complete["year"] == 11)]
year_twelve = school_data_complete[(school_data_complete["year"] == 12)]

# Group each by school name
year_nine_scores = year_nine.groupby(["school_name"])["maths_score"].mean()
year_ten_scores = year_ten.groupby(["school_name"])["maths_score"].mean()
year_eleven_scores = year_eleven.groupby(["school_name"])["maths_score"].mean()
year_twelve_scores = year_twelve.groupby(["school_name"])["maths_score"].mean()

# Combine series into single DataFrame
maths_scores_by_year = pd.DataFrame({"Year 9": year_nine_scores, "Year 10": year_ten_scores,
                                "Year 11": year_eleven_scores, "Year 12": year_twelve_scores})

# Minor data wrangling
maths_scores_by_year = maths_scores_by_year[["Year 9", "Year 10", "Year 11", "Year 12"]]
maths_scores_by_year.index.name = None

# Display the DataFrame
maths_scores_by_year


,Year 9,Year 10,Year 11,Year 12
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393
Pena High School,71.996364,72.396000,72.523438,71.187845


## Reading Score by Year

* Perform the same operations as above for reading scores

In [25]:
# Create data series of scores by year levels using conditionals
year_nine = school_data_complete[(school_data_complete["year"] == 9)]
year_ten = school_data_complete[(school_data_complete["year"] == 10)]
year_eleven = school_data_complete[(school_data_complete["year"] == 11)]
year_twelve = school_data_complete[(school_data_complete["year"] == 12)]

# Group each by school name
year_nine_scores = year_nine.groupby(["school_name"])["reading_score"].mean()
year_ten_scores = year_ten.groupby(["school_name"])["reading_score"].mean()
year_eleven_scores = year_eleven.groupby(["school_name"])["reading_score"].mean()
year_twelve_scores = year_twelve.groupby(["school_name"])["reading_score"].mean()

# Combine series into single DataFrame
reading_scores_by_year = pd.DataFrame({"Year 9": year_nine_scores, "Year 10": year_ten_scores,
                                "Year 11": year_eleven_scores, "Year 12": year_twelve_scores})

# Minor data wrangling
reading_scores_by_year = reading_scores_by_year[["Year 9", "Year 10", "Year 11", "Year 12"]]
reading_scores_by_year.index.name = None

# Display the DataFrame
reading_scores_by_year


,Year 9,Year 10,Year 11,Year 12
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521
Pena High School,70.949091,72.324000,71.703125,71.513812


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [26]:
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,Government,4976,3124928,628.0,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,1081356,582.0,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,1884411,639.0,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,1763916,644.0,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,917500,625.0,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,3022020,652.0,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,248087,581.0,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,1910635,655.0,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,3094650,650.0,68.843100,69.039277,82.062592,81.978576,67.191766


In [27]:
# create spending bins
bins = [0, 584.9, 629.9, 644.9, 680]
spending_range = ["<$585", "$585-630", "$630-645", "$645-680"]

spending_summary = school_summary.copy()

spending_summary["Spending Ranges (Per Student)"] = pd.cut(school_summary["Per Student Budget"], bins, labels=spending_range, include_lowest=True)

spending_maths_scores = spending_summary.groupby(["Spending Ranges (Per Student)"])["Average Maths Score"].mean()
spending_reading_scores = spending_summary.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_maths = spending_summary.groupby(["Spending Ranges (Per Student)"])["% Passing Maths"].mean()
spending_passing_reading = spending_summary.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = spending_summary.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

#place in dataframe            
spending_summary = pd.DataFrame({
    "Average Maths Score": spending_maths_scores,
    "Average Reading Score": spending_reading_scores,
    "% Passing Maths": spending_passing_maths,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending
            
})
            
#reorder columns
spending_summary = spending_summary[[
    "Average Maths Score",
    "Average Reading Score",
    "% Passing Maths",
    "% Passing Reading",
    "% Overall Passing"
]]

spending_summary.index.name = "Spending Ranges (Per Student)"

#formatting
spending_summary.style.format({'Average Maths Score': '{:.2f}', 
                              'Average Reading Score': '{:.2f}', 
                              '% Passing Maths': '{:.2f}', 
                              '% Passing Reading':'{:.2f}', 
                              '% Overall Passing': '{:.2f}'})

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.36,70.72,88.84,86.39,76.72
$585-630,72.07,71.03,91.52,87.29,79.88
$630-645,69.85,69.84,84.69,83.76,71.00
$645-680,68.88,69.05,81.57,81.77,66.76


## Scores by School Size

* Perform the same operations as above, based on school size.

In [28]:
#create size bins 
size_bins = [0, 999.9, 1999.9, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

per_school_summary = school_summary.copy()
per_school_summary["School Size"] = pd.cut(school_summary["Total Students"], bins=size_bins, labels=size_labels, include_lowest=True)

school_size_maths_scores = per_school_summary.groupby(["School Size"])["Average Maths Score"].mean()
school_size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
school_size_passing_maths = per_school_summary.groupby(["School Size"])["% Passing Maths"].mean()
school_size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
school_size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

#place in dataframe            
per_school_summary = pd.DataFrame({
    "Average Maths Score": school_size_maths_scores,
    "Average Reading Score": school_size_reading_scores,
    "% Passing Maths": school_size_passing_maths,
    "% Passing Reading": school_size_passing_reading,
    "% Overall Passing": school_size_overall_passing
            
})
            
#reorder columns
per_school_summary = per_school_summary[[
    "Average Maths Score",
    "Average Reading Score",
    "% Passing Maths",
    "% Passing Reading",
    "% Overall Passing"
]]

per_school_summary.index.name = "School Size"

#formatting
per_school_summary.style.format({'Average Maths Score': '{:.6f}', 
                              'Average Reading Score': '{:.6f}', 
                              '% Passing Maths': '{:.6f}', 
                              '% Passing Reading':'{:.6f}', 
                              '% Overall Passing': '{:.6f}'})

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


## Scores by School Type

* Perform the same operations as above, based on school type

In [29]:
#group by school type
per_school_summary = school_summary.copy()

school_type_maths_scores = per_school_summary.groupby(["School Type"])["Average Maths Score"].mean()
school_type_reading_scores = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
school_type_passing_maths = per_school_summary.groupby(["School Type"])["% Passing Maths"].mean()
school_type_passing_reading = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
school_type_overall_passing = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

#place in dataframe            
per_school_summary = pd.DataFrame({
    "Average Maths Score": school_type_maths_scores,
    "Average Reading Score": school_type_reading_scores,
    "% Passing Maths": school_type_passing_maths,
    "% Passing Reading": school_type_passing_reading,
    "% Overall Passing": school_type_overall_passing
            
})
            
#reorder columns
per_school_summary = per_school_summary[[
    "Average Maths Score",
    "Average Reading Score",
    "% Passing Maths",
    "% Passing Reading",
    "% Overall Passing"
]]

per_school_summary.index.name = "School Type"

#formatting
per_school_summary.style.format({'Average Maths Score': '{:.6f}', 
                              'Average Reading Score': '{:.6f}', 
                              '% Passing Maths': '{:.6f}', 
                              '% Passing Reading':'{:.6f}', 
                              '% Overall Passing': '{:.6f}'})


,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
